In [120]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



**This script performs a correlation between two implicit (click-though data) and explicit (human relevance judgement) feedback for an Enterprise Search (ES) service that uses Learning to Rank (LTR)**

In [121]:
# Much of the code in this notebook inspired by 
# https://towardsdatascience.com/compare-multiple-frequency-distributions-to-extract-valuable-information-from-a-dataset-10cba801f07b
# 

In [122]:
#!pip3 install -U datapane >/dev/null 2>&1

In [123]:
#!pip3 install -U jinja2

In [124]:
import seaborn as sns
import altair as alt 
import pandas as pd


exim=pd.read_csv("/kaggle/input/explicitimplicitv5/explicit-implicit-v5.csv")



In [125]:
exim.describe

In [126]:
import numpy as np
#Replace ‘R’ value into zero 

#exim['HUMAN'] = exim['HUMAN'].replace(['R'],0)
#convertion of object data to numeric

exim["HUMAN"] = pd.to_numeric(exim["HUMAN"])
 
# create Condition

#eximconditions = [
#    (exim['HUMAN'] == 0),
#    (exim['HUMAN'] >= 1) & (exim['HUMAN'] <= 2),
#    (exim['HUMAN'] >= 4) & (exim['HUMAN'] <= 5),
#    (exim['HUMAN'] > 5) & (exim['HUMAN'] <= 10),
#    (exim['HUMAN'] > 10)
#    ]




In [127]:
#exim.info

In [128]:
import seaborn as sns

import matplotlib.pyplot as plt


In [129]:
#sns.stripplot(x = 'Pos', y = 'Height', data = wnba)
sns.set(rc={'figure.figsize':(15,10)})
sns.set(font_scale=2)

sns.stripplot(x = 'HUMAN', y = 'CLICK-THRU-6', data = exim).set(title='Strip Plot of raw Click-through versus human relevance judgements',xlabel='human relevance judgements',ylabel='raw Click-through 6 months')

In [130]:
sns.set(rc={'figure.figsize':(15,10)})
sns.set(font_scale=2)

sns.stripplot(x = 'HUMAN', y = 'CTR-6', data = exim).set(title='Strip Plot of CTR versus Human relevance judgements',xlabel='Human relevance judgements',ylabel='CTR %')

In [131]:
#sns.boxplot(x = 'Pos', y = 'Height', data = wnba)
#sns.boxplot(x = 'HUMAN', y = 'CLICK-THRU-6', data = exim)

In [132]:
#sns.set(rc={'figure.figsize':(11.7,8.27)})
#sns.boxplot(x = 'Pos', y = 'Height', data = wnba, whis=4)
sns.boxplot(x = 'HUMAN', y = 'CLICK-THRU-6', data = exim, whis=4)

In [133]:
sns.boxplot(x = 'HUMAN', y = 'CTR-6', data = exim, whis=4).set(title='Box Plot of CTR versus Human relevance judgements',xlabel='Human relevance judgements',ylabel='CTR %')

In [134]:
!pip3 install scipy

In [135]:
from scipy.stats import ttest_rel

In [136]:
#exim.rename(columns={'HUMAN':'','CTR-6:''})

In [137]:

# calculate the Pearson's correlation between two variables
from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr
from scipy.stats import spearmanr
# seed random number generator
seed(1)
# prepare data

In [138]:
data1 = exim['HUMAN']
data2 = exim['CLICK-THRU-6']

data1 = data1.dropna()
data2 = data2.dropna()

# calculate Pearson's correlation
corr, _ = pearsonr(data1, data2)
print('Pearsons correlation human v raw click-through: %.3f' % corr)

In [139]:
data1 = exim['HUMAN']
data2 = exim['CTR-6']

data1 = data1.dropna()
data2 = data2.dropna()

# calculate Pearson's correlation
corr, _ = pearsonr(data1, data2)
print('Pearsons correlation human v CTR : %.3f' % corr)

In [140]:
data1 = exim['HUMAN']
data2 = exim['CLICK-THRU-6']

data1 = data1.dropna()
data2 = data2.dropna()

from scipy.stats import spearmanr
corr, _ = spearmanr(data1, data2)
print('Spearmans correlation human v raw click-through: %.3f' % corr)

In [141]:
data1 = exim['HUMAN']
data2 = exim['CTR-6']

data1 = data1.dropna()
data2 = data2.dropna()

from scipy.stats import spearmanr
corr, _ = spearmanr(data1, data2)
print('Spearmans correlation human v raw click-through: %.3f' % corr)

In [142]:
# calculate spearman's correlation
coef, p = spearmanr(data1, data2)
print('Spearmans correlation coefficient: %.3f' % coef)
print ('p-value',p)
# interpret the significance
alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

In [143]:
#another way to report the results from the t-test by creating a violin plot:
sns.violinplot(x='HUMAN', y='CLICK-THRU-6', bw=10, data=exim)

In [144]:
#another way to report the results from the t-test by creating a violin plot:
sns.violinplot(x='HUMAN', y='CTR-6', bw=2, data=exim)